In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/belka/'

data  test.parquet.zip	train.parquet.zip


In [ ]:
# # Navigate to the directory containing the zip file
# train_zip_path = '/content/drive/MyDrive/Colab Notebooks/belka/train.parquet.zip'
# test_zip_path = '/content/drive/MyDrive/Colab Notebooks/belka/test.parquet.zip'

# # Unzip the files
# extract_path = '/content/drive/MyDrive/Colab Notebooks/belka/data/'

# # Unzip the file
# !unzip -q "{train_zip_path}" -d "{extract_path}"
# !unzip -q "{test_zip_path}" -d "{extract_path}"

In [ ]:
!pip install fastparquet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.7 MB/s eta 0:00:00


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mertbayraktar","key":"b12bfc42522d7f7a19ac34dbc140cce9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d antoninadolgorukova/belka-supplementary-calcs-and-data-for-ml

!unzip \*.zip -d ./data/

Dataset URL: https://www.kaggle.com/datasets/antoninadolgorukova/belka-supplementary-calcs-and-data-for-ml
License(s): Community Data License Agreement - Sharing - Version 1.0
 92% 136M/148M [00:00<00:00, 237MB/s]
100% 148M/148M [00:00<00:00, 222MB/s]
Archive:  belka-supplementary-calcs-and-data-for-ml.zip
  inflating: ./data/features/bb_mol_apfp_features.csv  
  inflating: ./data/features/bb_mol_phys_chem_features.csv  
  inflating: ./data/features/tam_sim_fmcs_bb_smiles.qs  
  inflating: ./data/for_sim_analysis/mol_test_features_ecfp.qs  
  inflating: ./data/for_sim_analysis/preds_XGb_280k_l547_lb472.qs  
  inflating: ./data/for_sim_analysis/train_test_valid_280k_ecfp_1024.qs  
  inflating: ./data/smiles/all_bb_smiles_by_bb.csv  
  inflating: ./data/smiles/bb_smiles_sdfset.sdf  
  inflating: ./data/smiles/bb_summary.qs  
  inflating: ./data/train subsets/BRD4_all_bind1_rand_bind0.qs  
  inflating: ./data/train subsets/HSA_all_bind1_rand_bind0.qs  
  inflating: ./data/train subsets/sE

In [ ]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [ ]:
pd.options.display.max_rows = 250
pd.options.display.max_columns = 250

In [ ]:
train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/belka/data/train.parquet')
test = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/belka/data/test.parquet')

In [ ]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,BRD4,0
1,1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,HSA,0
2,2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,sEH,0
3,3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,BRD4,0
4,4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,HSA,0


In [ ]:
bb_mol_apfp_features = pd.read_csv('/content/data/features/bb_mol_apfp_features.csv')
bb_mol_phys_chem_features = pd.read_csv('/content/data/features/bb_mol_phys_chem_features.csv')

In [ ]:
enc = {'l': 1, 'y': 2, '@': 3, '3': 4, 'H': 5, 'S': 6, 'F': 7, 'C': 8, 'r': 9, 's': 10, '/': 11, 'c': 12, 'o': 13,
           '+': 14, 'I': 15, '5': 16, '(': 17, '2': 18, ')': 19, '9': 20, 'i': 21, '#': 22, '6': 23, '8': 24, '4': 25, '=': 26,
           '1': 27, 'O': 28, '[': 29, 'D': 30, 'B': 31, ']': 32, 'N': 33, '7': 34, 'n': 35, '-': 36}


smiles = train[train['protein_name']=='BRD4']['molecule_smiles'].values
assert (smiles!=train[train['protein_name']=='HSA']['molecule_smiles'].values).sum() == 0
assert (smiles!=train[train['protein_name']=='sEH']['molecule_smiles'].values).sum() == 0
def encode_smile(smile):
    tmp = [enc[i] for i in smile]
    tmp = tmp + [0]*(142-len(tmp))
    return np.array(tmp).astype(np.uint8)

smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
smiles_enc = np.stack(smiles_enc)
train_enc = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
train_enc['bind1'] = train[train['protein_name']=='BRD4']['binds'].values
train_enc['bind2'] = train[train['protein_name']=='HSA']['binds'].values
train_enc['bind3'] = train[train['protein_name']=='sEH']['binds'].values

# Extract buildingblock3_smiles
buildingblock3_smiles = train[train['protein_name'] == 'BRD4']['buildingblock3_smiles'].values

# Add buildingblock3_smiles to train_enc
train_enc['buildingblock3_smiles'] = buildingblock3_smiles



smiles = test['molecule_smiles'].values

smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
smiles_enc = np.stack(smiles_enc)
test_enc = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])

100%|██████████| 1674896/1674896 [00:30<00:00, 54354.79it/s]


ValueError: Length of values (558859) does not match length of index (1674896)

In [ ]:
buildingblock3_smiles_test = test['buildingblock3_smiles'].values
test_enc['buildingblock3_smiles'] = buildingblock3_smiles_test

In [ ]:
test_enc.shape

(1674896, 143)

In [ ]:
train.shape, test.shape, train_enc.shape, test_enc.shape

((295246830, 7), (1674896, 6), (98415610, 146), (1674896, 143))

In [ ]:
len(buildingblock3_smiles)

98415610

In [ ]:
len(buildingblock3_smiles_test)

1674896

In [ ]:
## 1-) kzıın verisini ekle
## 2-) gereksiz sütunları drop et
## 3-) encode et
## 4-) kaggle yükle

In [ ]:
# Merge datasets on the 'SMILES' column
# merged_df1 = train.merge(bb_mol_phys_chem_features, left_on='buildingblock1_smiles', right_on='SMILES', suffixes=('', '_block1'))
# merged_df2 = merged_df1.merge(bb_mol_phys_chem_features, left_on='buildingblock2_smiles', right_on='SMILES', suffixes=('', '_block2'))
final_merged_df = train_enc.merge(bb_mol_phys_chem_features, left_on='buildingblock3_smiles', right_on='SMILES', suffixes=('', '_block3'))
final_merged_df_test = test_enc.merge(bb_mol_phys_chem_features, left_on='buildingblock3_smiles', right_on='SMILES', suffixes=('', '_block3'))

# Drop duplicate columns
# final_merged_df.drop(columns=['SMILES', 'SMILES_block2', 'SMILES_block3'], inplace=True)


In [ ]:
final_merged_df.head()

,enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,enc37,enc38,enc39,enc40,enc41,enc42,enc43,enc44,enc45,enc46,enc47,enc48,enc49,enc50,enc51,enc52,enc53,enc54,enc55,enc56,enc57,enc58,enc59,enc60,enc61,enc62,enc63,enc64,enc65,enc66,enc67,enc68,enc69,enc70,enc71,enc72,enc73,enc74,enc75,enc76,enc77,enc78,enc79,enc80,enc81,enc82,enc83,enc84,enc85,enc86,enc87,enc88,enc89,enc90,enc91,enc92,enc93,enc94,enc95,enc96,enc97,enc98,enc99,enc100,enc101,enc102,enc103,enc104,enc105,enc106,enc107,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3,buildingblock3_smiles,SMILES,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC
0,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,4,8,8,8,33,4,12,4,12,12,12,35,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Br.Br.NCC1CCCN1c1cccnn1,Br.Br.NCC1CCCN1c1cccnn1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
1,8,22,8,8,28,12,27,12,12,12,12,17,8,33,12,18,35,12,17,33,8,8,4,8,8,8,33,4,12,4,12,12,12,35,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Br.Br.NCC1CCCN1c1cccnn1,Br.Br.NCC1CCCN1c1cccnn1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
2,8,22,8,8,29,8,3,3,5,32,17,8,8,17,26,28,19,33,29,30,2,32,19,33,12,27,35,12,17,33,8,8,18,8,8,8,33,18,12,18,12,12,12,35,35,18,19,35,12,17,33,12,18,12,12,12,17,8,22,8,19,12,12,18,19,35,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Br.Br.NCC1CCCN1c1cccnn1,Br.Br.NCC1CCCN1c1cccnn1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
3,8,22,8,8,29,8,3,3,5,32,17,8,8,17,26,28,19,33,29,30,2,32,19,33,12,27,35,12,17,33,8,8,18,8,8,8,33,18,12,18,12,12,12,35,35,18,19,35,12,17,33,12,18,12,12,12,12,17,8,22,8,19,12,18,19,35,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Br.Br.NCC1CCCN1c1cccnn1,Br.Br.NCC1CCCN1c1cccnn1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
4,8,22,8,8,29,8,3,3,5,32,17,8,8,17,26,28,19,33,29,30,2,32,19,33,12,27,35,12,17,33,8,8,33,8,17,26,28,19,8,17,26,8,19,8,19,35,12,17,33,8,8,18,8,8,8,33,18,12,18,12,12,12,35,35,18,19,35,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Br.Br.NCC1CCCN1c1cccnn1,Br.Br.NCC1CCCN1c1cccnn1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1


In [ ]:
final_merged_df.shape, final_merged_df_test.shape, train_enc.shape, test_enc.shape, train.shape, test.shape

((98415610, 174),
 (1674896, 171),
 (98415610, 146),
 (1674896, 143),
 (295246830, 7),
 (1674896, 6))

In [ ]:
final_merged_df_test.head()

,enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,enc37,enc38,enc39,enc40,enc41,enc42,enc43,enc44,enc45,enc46,enc47,enc48,enc49,enc50,enc51,enc52,enc53,enc54,enc55,enc56,enc57,enc58,enc59,enc60,enc61,enc62,enc63,enc64,enc65,enc66,enc67,enc68,enc69,enc70,enc71,enc72,enc73,enc74,enc75,enc76,enc77,enc78,enc79,enc80,enc81,enc82,enc83,enc84,enc85,enc86,enc87,enc88,enc89,enc90,enc91,enc92,enc93,enc94,enc95,enc96,enc97,enc98,enc99,enc100,enc101,enc102,enc103,enc104,enc105,enc106,enc107,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,buildingblock3_smiles,SMILES,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC
0,8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,27,19,8,17,26,28,19,33,29,30,2,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
1,8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,27,19,8,17,26,28,19,33,29,30,2,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
2,8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,27,19,8,17,26,28,19,33,29,30,2,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
3,8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,8,8,8,18,17,8,28,8,19,8,8,18,19,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,27,19,8,17,26,28,19,33,29,30,2,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
4,8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,8,8,8,18,17,8,28,8,19,8,8,18,19,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,27,19,8,17,26,28,19,33,29,30,2,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1


In [ ]:
def print_column_ranges(df):
    int_columns = df.select_dtypes(include=['int64']).columns
    for col in int_columns:
        min_val = df[col].min()
        max_val = df[col].max()
        print(f"Column {col}: min = {min_val}, max = {max_val}")

# Assuming df is your DataFrame
print_column_ranges(final_merged_df)


Column bind1: min = 0, max = 1
Column bind2: min = 0, max = 1
Column bind3: min = 0, max = 1
Column Ncharges: min = 0, max = 2
Column C: min = 1, max = 15
Column H: min = 3, max = 23
Column N: min = 1, max = 6
Column O: min = 0, max = 4
Column Cl: min = 0, max = 3
Column F: min = 0, max = 6
Column S: min = 0, max = 2
Column Br: min = 0, max = 2
Column I: min = 0, max = 0
Column B: min = 0, max = 1
Column Si: min = 0, max = 1
Column RNH2: min = 0, max = 1
Column R2NH: min = 0, max = 2
Column R3N: min = 0, max = 2
Column ROH: min = 0, max = 1
Column RCHO: min = 0, max = 1
Column RCOR: min = 0, max = 1
Column RCOOH: min = 0, max = 2
Column RCOOR: min = 0, max = 2
Column ROR: min = 0, max = 3
Column RCCH: min = 0, max = 2
Column RCN: min = 0, max = 2
Column RINGS: min = 0, max = 27
Column AROMATIC: min = 0, max = 6


In [ ]:
final_merged_df.drop(columns=['buildingblock3_smiles', 'SMILES', 'MF'], inplace=True)
final_merged_df_test.drop(columns=['buildingblock3_smiles', 'SMILES', 'MF'], inplace=True)

In [ ]:
# Function to downcast integer columns to int8 or int16
def downcast_int_columns(df):
    int_columns = df.select_dtypes(include=['int64']).columns
    for col in int_columns:
          df[col] = df[col].astype('int8')
          print(f"Column {col} downcast to int8")
    return df

# Downcast integer columns in the DataFrame
final_merged_df = downcast_int_columns(final_merged_df)
final_merged_df_test = downcast_int_columns(final_merged_df_test)

Column bind1 downcast to int8
Column bind2 downcast to int8
Column bind3 downcast to int8
Column Ncharges downcast to int8
Column C downcast to int8
Column H downcast to int8
Column N downcast to int8
Column O downcast to int8
Column Cl downcast to int8
Column F downcast to int8
Column S downcast to int8
Column Br downcast to int8
Column I downcast to int8
Column B downcast to int8
Column Si downcast to int8
Column RNH2 downcast to int8
Column R2NH downcast to int8
Column R3N downcast to int8
Column ROH downcast to int8
Column RCHO downcast to int8
Column RCOR downcast to int8
Column RCOOH downcast to int8
Column RCOOR downcast to int8
Column ROR downcast to int8
Column RCCH downcast to int8
Column RCN downcast to int8
Column RINGS downcast to int8
Column AROMATIC downcast to int8
Column Ncharges downcast to int8
Column C downcast to int8
Column H downcast to int8
Column N downcast to int8
Column O downcast to int8
Column Cl downcast to int8
Column F downcast to int8
Column S downcast 

In [ ]:
final_merged_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674896 entries, 0 to 1674895
Columns: 168 entries, enc0 to AROMATIC
dtypes: float64(1), int8(25), uint8(142)
memory usage: 279.5 MB


In [ ]:
def downcast_float_columns(df):
    float_columns = df.select_dtypes(include=['float64']).columns
    for col in float_columns:
        df[col] = df[col].astype('float32')
        print(f"Column {col} downcast to float32")
    return df

# Apply the function to your DataFrame
final_merged_df = downcast_float_columns(final_merged_df)
final_merged_df_test = downcast_float_columns(final_merged_df_test)


Column MW downcast to float32
Column MW downcast to float32


In [ ]:
final_merged_df.to_parquet('train.parquet')
final_merged_df_test.to_parquet('test.parquet')

In [ ]:
## teste de antonina ekle

In [ ]:
# BBs_dict_reverse_1 = pickle.load(open('/content/data/train_dicts/BBs_dict_reverse_1.p', 'br'))
# BBs_dict_reverse_2 = pickle.load(open('/content/data/train_dicts/BBs_dict_reverse_2.p', 'br'))
# BBs_dict_reverse_3 = pickle.load(open('/content/data/train_dicts/BBs_dict_reverse_3.p', 'br'))

In [ ]:
# train.shape

(98415610, 7)

In [ ]:
# buildingblock3_smiles_original = [BBs_dict_reverse_3[x] for x in train.buildingblock3_smiles]
# print(buildingblock3_smiles_original[0])

Br.Br.NCC1CCCN1c1cccnn1


In [ ]:
# buildingblock2_smiles_original = [BBs_dict_reverse_2[x] for x in train.buildingblock2_smiles]
# print(buildingblock2_smiles_original[0])

C#CCOc1ccc(CN)cc1.Cl


In [ ]:
# buildingblock1_smiles_original = [BBs_dict_reverse_1[x] for x in train.buildingblock1_smiles]
# print(buildingblock1_smiles_original[0])

C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21


In [ ]:
# train['buildingblock3_smiles'] = buildingblock3_smiles_original
# train['buildingblock2_smiles'] = buildingblock2_smiles_original
# train['buildingblock1_smiles'] = buildingblock1_smiles_original

In [ ]:
# train.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,binds_BRD4,binds_HSA,binds_sEH
0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,0,0,0
1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,0,0,0
2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,C#CCOc1ccc(CN)cc1.Cl,C#CCOc1ccc(CNc2nc(NCc3ccc(OCC#C)cc3)nc(N[C@@H]...,0,0,0
3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,C=C(C)C(=O)NCCN.Cl,C#CCOc1ccc(CNc2nc(NCCNC(=O)C(=C)C)nc(N[C@@H](C...,0,0,0
4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,C=CCNC(=O)CN.Cl,C#CCOc1ccc(CNc2nc(NCC(=O)NCC=C)nc(N[C@@H](CC#C...,0,0,0


In [ ]:
bb_mol_phys_chem_features.shape

(2110, 28)

In [ ]:
test.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,BRD4
1,295246831,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,HSA
2,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,sEH
3,295246833,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,BRD4
4,295246834,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,HSA


In [ ]:
test.shape

(1674896, 6)

In [ ]:
# # Merge datasets on the 'SMILES' column
# merged_df1 = train.merge(bb_mol_phys_chem_features, left_on='buildingblock1_smiles', right_on='SMILES', suffixes=('', '_block1'))
# merged_df2 = merged_df1.merge(bb_mol_phys_chem_features, left_on='buildingblock2_smiles', right_on='SMILES', suffixes=('', '_block2'))
# final_merged_df = merged_df2.merge(bb_mol_phys_chem_features, left_on='buildingblock3_smiles', right_on='SMILES', suffixes=('', '_block3'))

# # Drop duplicate columns
# final_merged_df.drop(columns=['SMILES', 'SMILES_block2', 'SMILES_block3'], inplace=True)


In [ ]:
# BBs_dict_reverse_1 = pickle.load(open('/content/data/test_dicts/BBs_dict_reverse_1_test.p', 'br'))
# BBs_dict_reverse_2 = pickle.load(open('/content/data/test_dicts/BBs_dict_reverse_2_test.p', 'br'))
# BBs_dict_reverse_3 = pickle.load(open('/content/data/test_dicts/BBs_dict_reverse_3_test.p', 'br'))

In [ ]:
# buildingblock3_smiles_original = [BBs_dict_reverse_3[x] for x in test.buildingblock3_smiles]
# print(buildingblock3_smiles_original[1])

C=Cc1ccc(N)cc1


In [ ]:
# buildingblock2_smiles_original = [BBs_dict_reverse_2[x] for x in test.buildingblock2_smiles]
# print(buildingblock2_smiles_original[1])

C=Cc1ccc(N)cc1


In [ ]:
# buildingblock1_smiles_original = [BBs_dict_reverse_1[x] for x in test.buildingblock1_smiles]
# print(buildingblock1_smiles_original[1])

C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O


In [ ]:
# test['buildingblock3_smiles'] = buildingblock3_smiles_original
# test['buildingblock2_smiles'] = buildingblock2_smiles_original
# test['buildingblock1_smiles'] = buildingblock1_smiles_original

In [ ]:
# # Merge datasets on the 'SMILES' column
# merged_df1_test = test.merge(bb_mol_phys_chem_features, left_on='buildingblock1_smiles', right_on='SMILES', suffixes=('', '_block1'))
# merged_df2_test = merged_df1_test.merge(bb_mol_phys_chem_features, left_on='buildingblock2_smiles', right_on='SMILES', suffixes=('', '_block2'))
# final_merged_df_test = merged_df2_test.merge(bb_mol_phys_chem_features, left_on='buildingblock3_smiles', right_on='SMILES', suffixes=('', '_block3'))

# # Drop duplicate columns
# final_merged_df_test.drop(columns=['SMILES', 'SMILES_block2', 'SMILES_block3'], inplace=True)


In [ ]:
# Create the new columns and populate them with 0 or 1
final_merged_df_test['binds_BRD4'] = final_merged_df_test['protein_name'].apply(lambda x: 1 if x == 'BRD4' else 0)
final_merged_df_test['binds_HSA'] = final_merged_df_test['protein_name'].apply(lambda x: 1 if x == 'HSA' else 0)
final_merged_df_test['binds_sEH'] = final_merged_df_test['protein_name'].apply(lambda x: 1 if x == 'sEH' else 0)

In [ ]:
final_merged_df_test.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3,binds_BRD4,binds_HSA,binds_sEH
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,BRD4,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
1,295246831,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,HSA,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,sEH,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
3,295568792,CC(C)(C)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3cc...,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C=Cc1ccc(Nc2nc(Nc3ccc(C=C)cc3)nc(Nc3cc(C(C)(C)...,BRD4,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,295568793,CC(C)(C)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3cc...,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C=Cc1ccc(Nc2nc(Nc3ccc(C=C)cc3)nc(Nc3cc(C(C)(C)...,HSA,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [ ]:
final_merged_df_test = final_merged_df_test.drop(['protein_name'], axis=1)
final_merged_df_test = final_merged_df_test.drop(['id'], axis=1)

In [ ]:
final_merged_df_test.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3,binds_BRD4,binds_HSA,binds_sEH
0,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
1,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
3,CC(C)(C)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3cc...,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C=Cc1ccc(Nc2nc(Nc3ccc(C=C)cc3)nc(Nc3cc(C(C)(C)...,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,CC(C)(C)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3cc...,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C=Cc1ccc(Nc2nc(Nc3ccc(C=C)cc3)nc(Nc3cc(C(C)(C)...,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [ ]:
train.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,binds_BRD4,binds_HSA,binds_sEH,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3
0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
1,C#CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@H](...,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
2,C#CC[C@@](C)(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@](C...,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
3,C#CC[C@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@H](...,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
4,C#CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1


In [ ]:
bb_mol_phys_chem_features.head()

,SMILES,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC
0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2
1,C#CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2
2,C=CCC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C21H21NO4,351.39574,0,21,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,6,2
3,CC(C)(C)OC(=O)CCC(NC(=O)OCC1c2ccccc2-c2ccccc21...,C24H27NO6,425.47428,0,24,27,1,6,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,0,0,0,6,2
4,CC(C)(C)OC(=O)N1CCN(C(=O)OCC2c3ccccc3-c3ccccc3...,C24H26N2O6,438.47304,0,24,26,2,6,0,0,0,0,0,0,0,0,0,2,0,0,0,1,2,0,0,0,7,2


In [ ]:
final_merged_df_test.shape

(1674896, 88)

In [ ]:
final_merged_df_test.to_parquet('final_merged_df_test.parquet')

In [ ]:
train.shape

(98415610, 88)

In [ ]:
!kaggle datasets download -d mertbayraktar/belka-shrunken-additional-features

Dataset URL: https://www.kaggle.com/datasets/mertbayraktar/belka-shrunken-additional-features
License(s): unknown
 98% 949M/964M [00:08<00:00, 97.0MB/s]
100% 964M/964M [00:09<00:00, 112MB/s] 


In [ ]:
!unzip \*.zip -d ./data/

Archive:  belka-shrunken-additional-features.zip
  inflating: ./data/final_merged_df.parquet  
  inflating: ./data/final_merged_df_test.parquet  

Archive:  belka-shrunken-train-set.zip
replace ./data/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./data/test.csv         
  inflating: ./data/test.parquet     
  inflating: ./data/test_dicts/BBs_dict_1_test.p  
  inflating: ./data/test_dicts/BBs_dict_2_test.p  
  inflating: ./data/test_dicts/BBs_dict_3_test.p  
  inflating: ./data/test_dicts/BBs_dict_reverse_1_test.p  
  inflating: ./data/test_dicts/BBs_dict_reverse_2_test.p  
  inflating: ./data/test_dicts/BBs_dict_reverse_3_test.p  
  inflating: ./data/test_dicts/molecule_smiles_unique.p  
  inflating: ./data/train.csv        
  inflating: ./data/train.parquet    
  inflating: ./data/train_dicts/BBs_dict_1.p  
  inflating: ./data/train_dicts/BBs_dict_2.p  
  inflating: ./data/train_dicts/BBs_dict_3.p  
  inflating: ./data/train_dicts/BBs_dict_reverse_1.p  
  inflating: 

In [ ]:
train = pd.read_parquet('/content/data/final_merged_df.parquet')
test = pd.read_parquet('/content/data/final_merged_df_test.parquet')

In [ ]:
BBs_dict_1 = pickle.load(open('/content/data/train_dicts/BBs_dict_1.p', 'br'))
BBs_dict_2 = pickle.load(open('/content/data/train_dicts/BBs_dict_2.p', 'br'))
BBs_dict_3 = pickle.load(open('/content/data/train_dicts/BBs_dict_3.p', 'br'))

In [ ]:
buildingblock3_smiles = [BBs_dict_3[x] for x in train.buildingblock3_smiles]
print(buildingblock3_smiles[0])

buildingblock2_smiles = [BBs_dict_2[x] for x in train.buildingblock2_smiles]
print(buildingblock2_smiles[0])

buildingblock1_smiles = [BBs_dict_1[x] for x in train.buildingblock1_smiles]
print(buildingblock1_smiles[0])

0
0
0


In [ ]:
train['buildingblock3_smiles'] = buildingblock3_smiles
train['buildingblock2_smiles'] = buildingblock2_smiles
train['buildingblock1_smiles'] = buildingblock1_smiles

In [ ]:
train.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,binds_BRD4,binds_HSA,binds_sEH,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3
0,0,0,0,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
1,1,0,0,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@H](...,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
2,2,0,0,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@](C...,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
3,3,0,0,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@H](...,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
4,4,0,0,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1


In [ ]:
train.shape

(98415610, 88)

In [ ]:
train = train.drop(['molecule_smiles'], axis=1)

In [ ]:
BBs_dict_1_test = pickle.load(open('/content/data/test_dicts/BBs_dict_1_test.p', 'br'))
BBs_dict_2_test = pickle.load(open('/content/data/test_dicts/BBs_dict_2_test.p', 'br'))
BBs_dict_3_test = pickle.load(open('/content/data/test_dicts/BBs_dict_3_test.p', 'br'))

In [ ]:
buildingblock3_smiles_test = [BBs_dict_3_test[x] for x in test.buildingblock3_smiles]
print(buildingblock3_smiles_test[0])

buildingblock2_smiles_test = [BBs_dict_2_test[x] for x in test.buildingblock2_smiles]
print(buildingblock2_smiles_test[0])

buildingblock1_smiles_test = [BBs_dict_1_test[x] for x in test.buildingblock1_smiles]
print(buildingblock1_smiles_test[0])

17
17
0


In [ ]:
test['buildingblock3_smiles'] = buildingblock3_smiles_test
test['buildingblock2_smiles'] = buildingblock2_smiles_test
test['buildingblock1_smiles'] = buildingblock1_smiles_test

In [ ]:
test.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3,binds_BRD4,binds_HSA,binds_sEH
0,0,17,17,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
1,0,17,17,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2,0,17,17,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
3,36,17,17,C=Cc1ccc(Nc2nc(Nc3ccc(C=C)cc3)nc(Nc3cc(C(C)(C)...,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,36,17,17,C=Cc1ccc(Nc2nc(Nc3ccc(C=C)cc3)nc(Nc3cc(C(C)(C)...,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [ ]:
test.shape

(1674896, 88)

In [ ]:
test = test.drop(['molecule_smiles'], axis=1)

In [ ]:
test.to_parquet("test_enc.parquet")
train.to_parquet("train_enc.parquet")

In [ ]:
train.shape

(98415610, 87)

In [ ]:
train.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,binds_BRD4,binds_HSA,binds_sEH,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3
0,0,0,0,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
1,1,0,0,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
2,2,0,0,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
3,3,0,0,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
4,4,0,0,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1


In [ ]:
test.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3,binds_BRD4,binds_HSA,binds_sEH
0,0,17,17,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
1,0,17,17,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2,0,17,17,C22H21NO4,363.40644,0,22,21,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
3,36,17,17,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,36,17,17,C26H25NO4,415.48100,0,26,25,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,7,3,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,C8H9N,119.16376,0,8,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
